In [1]:
import os
import numpy as np
import pandas as pd
import itertools
from IPython.display import display
%matplotlib inline
os.chdir("..")
home_dir = os.getcwd()
os.chdir("src")
from smartsifter import SmartSifter
os.chdir(home_dir)

In [2]:
org_data = pd.read_csv("data/kddcup.data.corrected")
header_data = pd.read_csv("data/header.csv")
display(header_data)

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,...,dst_host_count,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate


In [3]:
header = list(header_data.columns)
header.append("label")
org_data.columns = header

In [4]:
display(org_data.head())
display(org_data.tail())
print(org_data.shape)

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,...,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,label
0,0,tcp,http,SF,162,4528,0,0,0,0,...,1,1.0,0.0,1.00,0.0,0.0,0.0,0.0,0.0,normal.
1,0,tcp,http,SF,236,1228,0,0,0,0,...,2,1.0,0.0,0.50,0.0,0.0,0.0,0.0,0.0,normal.
2,0,tcp,http,SF,233,2032,0,0,0,0,...,3,1.0,0.0,0.33,0.0,0.0,0.0,0.0,0.0,normal.
3,0,tcp,http,SF,239,486,0,0,0,0,...,4,1.0,0.0,0.25,0.0,0.0,0.0,0.0,0.0,normal.
4,0,tcp,http,SF,238,1282,0,0,0,0,...,5,1.0,0.0,0.20,0.0,0.0,0.0,0.0,0.0,normal.


,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,...,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,label
4898425,0,tcp,http,SF,212,2288,0,0,0,0,...,255,1.0,0.0,0.33,0.05,0.0,0.01,0.0,0.0,normal.
4898426,0,tcp,http,SF,219,236,0,0,0,0,...,255,1.0,0.0,0.25,0.05,0.0,0.01,0.0,0.0,normal.
4898427,0,tcp,http,SF,218,3610,0,0,0,0,...,255,1.0,0.0,0.20,0.05,0.0,0.01,0.0,0.0,normal.
4898428,0,tcp,http,SF,219,1234,0,0,0,0,...,255,1.0,0.0,0.17,0.05,0.0,0.01,0.0,0.0,normal.
4898429,0,tcp,http,SF,219,1098,0,0,0,0,...,255,1.0,0.0,0.14,0.05,0.0,0.01,0.0,0.0,normal.


(4898430, 42)


In [5]:
df = org_data.copy()
display(df.head())

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,...,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,label
0,0,tcp,http,SF,162,4528,0,0,0,0,...,1,1.0,0.0,1.00,0.0,0.0,0.0,0.0,0.0,normal.
1,0,tcp,http,SF,236,1228,0,0,0,0,...,2,1.0,0.0,0.50,0.0,0.0,0.0,0.0,0.0,normal.
2,0,tcp,http,SF,233,2032,0,0,0,0,...,3,1.0,0.0,0.33,0.0,0.0,0.0,0.0,0.0,normal.
3,0,tcp,http,SF,239,486,0,0,0,0,...,4,1.0,0.0,0.25,0.0,0.0,0.0,0.0,0.0,normal.
4,0,tcp,http,SF,238,1282,0,0,0,0,...,5,1.0,0.0,0.20,0.0,0.0,0.0,0.0,0.0,normal.


In [6]:
seed = 0
SF = df.loc[df.logged_in > 0]
SF10 = SF.sample(n=100000, random_state=seed)
SF10.shape

(100000, 42)

In [7]:
dataset = SF10[["duration", "src_bytes", "dst_bytes", "label"]]
display(dataset.head())
display(dataset.tail())
display(dataset.describe())

for i in dataset.columns[:-1]:
    dataset[i] = np.log(dataset[i] + 0.1)
    
display(dataset.head())
display(dataset.tail())
display(dataset.describe())

,duration,src_bytes,dst_bytes,label
3429021,3,866,327,normal.
720878,74,208,2719,normal.
299626,0,249,6185,normal.
10954,0,287,2536,normal.
497911,1,912,329,normal.


,duration,src_bytes,dst_bytes,label
4554267,0,276,313,normal.
6675,0,203,4624,normal.
1368339,0,346,6388,normal.
22059,0,317,1856,normal.
275386,0,305,1304,normal.


,duration,src_bytes,dst_bytes
count,100000.000000,1.000000e+05,1.000000e+05
mean,12.179180,1.749763e+03,4.583439e+03
std,384.622732,5.347187e+04,4.019262e+04
min,0.000000,0.000000e+00,0.000000e+00
25%,0.000000,2.220000e+02,3.460000e+02
50%,0.000000,2.820000e+02,1.192000e+03
75%,0.000000,3.330000e+02,3.222000e+03
max,29296.000000,5.135678e+06,4.091815e+06


/Users/satetsu/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,duration,src_bytes,dst_bytes,label
3429021,1.131402,6.764000,5.790266,normal.
720878,4.305416,5.338019,7.908056,normal.
299626,-2.302585,5.517854,8.729898,normal.
10954,-2.302585,5.659831,7.838383,normal.
497911,0.095310,6.815750,5.796362,normal.


,duration,src_bytes,dst_bytes,label
4554267,-2.302585,5.620763,5.746523,normal.
6675,-2.302585,5.313698,8.439037,normal.
1368339,-2.302585,5.846728,8.762192,normal.
22059,-2.302585,5.759217,7.526233,normal.
275386,-2.302585,5.720640,7.173268,normal.


,duration,src_bytes,dst_bytes
count,100000.000000,100000.000000,100000.000000
mean,-1.991613,5.853197,6.823425
std,1.132627,0.901850,2.397276
min,-2.302585,-2.302585,-2.302585
25%,-2.302585,5.403128,5.846728
50%,-2.302585,5.642262,7.083472
75%,-2.302585,5.808443,8.077789
max,10.285210,15.451722,15.224499


In [8]:
x = np.array([0 for i in range(dataset.shape[0])])
y = np.array(dataset.iloc[:, :-1])
K = 2

In [ ]:
#パラメータの設定
np.random.seed(seed)
#SDLE
A = list(itertools.product(set(x)))
r = 0.0002 #忘却パラメータ
beta = 1 #正の定数

#SDEM
alpha = 2.0 #(1.0~2.0)
r = r
k = K #潜在変数の数
# calc_helinger_score
r_h = 0.0003

In [ ]:
ss = SmartSifter(r, beta, A, alpha, k, r_h)
ss.train(x, y, show=True)

/Users/satetsu/program/python/datamining/anomaly_detection_with_data_mining/src/smartsifter.py:76: RuntimeWarning: divide by zero encountered in log
  s_l = -np.log(p_prev_params) #シャノン情報量を計算


calculated: 1.0%
calculated: 2.0%
calculated: 3.0%
calculated: 4.0%
calculated: 5.0%
calculated: 6.0%
calculated: 7.000000000000001%
calculated: 8.0%
calculated: 9.0%
calculated: 10.0%
calculated: 11.0%
calculated: 12.0%
calculated: 13.0%
calculated: 14.000000000000002%
calculated: 15.0%
calculated: 16.0%
calculated: 17.0%
calculated: 18.0%
calculated: 19.0%
calculated: 20.0%
calculated: 21.0%
calculated: 22.0%
calculated: 23.0%
calculated: 24.0%
calculated: 25.0%
calculated: 26.0%
calculated: 27.0%
calculated: 28.000000000000004%
calculated: 28.999999999999996%
calculated: 30.0%
calculated: 31.0%
calculated: 32.0%
calculated: 33.0%
calculated: 34.0%
calculated: 35.0%
calculated: 36.0%
calculated: 37.0%
calculated: 38.0%
calculated: 39.0%
calculated: 40.0%
calculated: 41.0%
calculated: 42.0%
calculated: 43.0%
calculated: 44.0%
calculated: 45.0%
calculated: 46.0%
calculated: 47.0%
calculated: 48.0%
calculated: 49.0%
calculated: 50.0%
calculated: 51.0%
calculated: 52.0%
calculated: 53.0%

In [ ]:
dataset["S_L"] = np.array(ss.S_L)
dataset["S_H"] = np.array(ss.S_H)[:, 0, 0]
# dataset.to_csv("data/SF10.csv")
dataset.describe()

In [ ]:
top10_S_L = dataset.sort(columns="S_L", ascending=False)[:10000]
ratio_s_l = top10_S_L.loc[top10_S_L.label != "normal."].shape[0]/dataset.loc[dataset.label != "normal."].shape[0]
top10_S_H = dataset.sort(columns="S_H", ascending=False)[:10000]
ratio_s_h = top10_S_H.loc[top10_S_H.label != "normal."].shape[0]/dataset.loc[dataset.label != "normal."].shape[0]
print(ratio_s_l, ratio_s_h)

In [ ]:
top20_S_L = dataset.sort(columns="S_L", ascending=False)[:20000]
ratio_s_l = top20_S_L.loc[top20_S_L.label != "normal."].shape[0]/dataset.loc[dataset.label != "normal."].shape[0]
top20_S_H = dataset.sort(columns="S_H", ascending=False)[:20000]
ratio_s_h = top20_S_H.loc[top20_S_H.label != "normal."].shape[0]/dataset.loc[dataset.label != "normal."].shape[0]
print(ratio_s_l, ratio_s_h)